In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [ ]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [ ]:
# 3. 최신 변경사항 가져오기
!git pull origin main

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 1.96 KiB | 1024 bytes/s, done.
From https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project
 * branch            main       -> FETCH_HEAD
   57fbabb..e460a65  main       -> origin/main
Updating 57fbabb..e460a65
Fast-forward
 .gitignore | 5 ++++-
 1 file changed, 4 insertions(+), 1 deletion(-)


In [ ]:
# 3. 현재 상태 확인
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/data_EDA.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data_pipeline/notebooks/
	notebooks/New_split_dataset.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


# DETR 전용 데이터 전처리 코드

## 초기 설정

In [ ]:
import os
import json
import shutil
from collections import defaultdict
from datetime import datetime
import glob
from PIL import Image

In [ ]:
class DETR_preprocessor:
    def __init__(self, data_path = "./data"):
        self.data_path = data_path
        self.test_images_path = os.path.join(data_path, "test_images")
        self.train_images_path = os.path.join(data_path, "train_images")
        self.train_ann_path = os.path.join(data_path, "train_annotations")

        # 출력 디렉토리
        self.output_dir = os.path.join(data_path, "processed")
        os.makedirs(self.output_dir, exist_ok=True)

    def find_all_json_files(self):
        """
        train_annotations 디렉토리에서 모든 JSON 파일을 찾기
        """
        pattern = os.path.join(self.train_ann_path, "**", "*.json")
        json_files = glob.glob(pattern, recursive=True)

        print(f"총 {len(json_files)}개의 JSON 파일을 찾았습니다.")
        return json_files

    # DETR은 하나의 JSON파일을 사용한다.
    def merge_coco_annotations(self, json_files):
        """
        여러 COCO 포맷 JSON 파일들을 하나로 병합
        - 동일한 알약의 다른 인스턴스들은 모두 보존
        - 카테고리는 이름 기준으로 통일
        """
        merged_data = {
            "images": [],
            "annotations": [],
            "categories": [],
            "info": {
                "description": "Merged Pill Dataset",
                "version": "1.0",
                "year": 2024,
                "date_created": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
        }

        # ID 재할당을 위한 카운터
        new_image_id = 1
        new_annotation_id = 1

        # 카테고리 통합 (이름 기준)
        category_name_to_new_id = {}
        new_category_id = 1

        # 중복 이미지 파일 추적 (파일명 기준)
        processed_filenames = set()

        print("COCO 포맷 JSON 파일들을 병합하는 중...")

        for i, json_file in enumerate(json_files):
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                print(f"처리 중: {os.path.basename(json_file)} ({i+1}/{len(json_files)})")

                # 1. 카테고리 병합 및 ID 매핑 생성
                original_to_new_category_id = {}

                if 'categories' in data:
                    for category in data['categories']:
                        cat_name = category['name']
                        original_cat_id = category['id']

                        if cat_name not in category_name_to_new_id:
                            # 새로운 카테고리 추가
                            new_category = {
                                "id": new_category_id,
                                "name": cat_name,
                                "supercategory": category.get('supercategory', 'pill')
                            }
                            merged_data['categories'].append(new_category)
                            category_name_to_new_id[cat_name] = new_category_id
                            original_to_new_category_id[original_cat_id] = new_category_id
                            new_category_id += 1
                        else:
                            # 기존 카테고리 ID 매핑
                            original_to_new_category_id[original_cat_id] = category_name_to_new_id[cat_name]

                # 2. 이미지 처리 및 ID 매핑 생성
                original_to_new_image_id = {}

                if 'images' in data:
                    for image in data['images']:
                        filename = image['file_name']
                        original_img_id = image['id']

                        # 이미지 파일 존재 확인
                        img_path = os.path.join(self.train_images_path, filename)
                        if not os.path.exists(img_path):
                            print(f"경고: 이미지 파일이 없습니다 - {filename}")
                            continue

                        # 파일명 중복 체크 (같은 파일은 한 번만 추가)
                        if filename in processed_filenames:
                            print(f"중복 파일명 스킵: {filename}")
                            # 하지만 ID 매핑은 유지해야 함
                            for existing_img in merged_data['images']:
                                if existing_img['file_name'] == filename:
                                    original_to_new_image_id[original_img_id] = existing_img['id']
                                    break
                            continue

                        # 새로운 이미지 추가
                        new_image = image.copy()
                        new_image['id'] = new_image_id
                        merged_data['images'].append(new_image)

                        original_to_new_image_id[original_img_id] = new_image_id
                        processed_filenames.add(filename)
                        new_image_id += 1

                # 3. 어노테이션 처리 (모든 인스턴스 보존)
                if 'annotations' in data:
                    for annotation in data['annotations']:
                        original_img_id = annotation['image_id']
                        original_cat_id = annotation['category_id']

                        # ID 매핑 확인
                        if (original_img_id in original_to_new_image_id and
                            original_cat_id in original_to_new_category_id):

                            new_annotation = annotation.copy()
                            new_annotation['id'] = new_annotation_id
                            new_annotation['image_id'] = original_to_new_image_id[original_img_id]
                            new_annotation['category_id'] = original_to_new_category_id[original_cat_id]

                            merged_data['annotations'].append(new_annotation)
                            new_annotation_id += 1
                        else:
                            print(f"경고: 매핑되지 않은 어노테이션 - img_id: {original_img_id}, cat_id: {original_cat_id}")

            except Exception as e:
                print(f"오류 발생 (파일: {json_file}): {e}")
                continue

        print(f"\n병합 완료:")
        print(f"  - 이미지: {len(merged_data['images'])}개")
        print(f"  - 어노테이션: {len(merged_data['annotations'])}개")
        print(f"  - 카테고리: {len(merged_data['categories'])}개")

        return merged_data

    def validate_merged_data(self, merged_data):
        """
        병합된 데이터의 무결성 검증
        """
        print("\n=== 데이터 무결성 검증 ===")

        # 이미지 ID 중복 체크
        image_ids = [img['id'] for img in merged_data['images']]
        if len(image_ids) != len(set(image_ids)):
            print("경고: 이미지 ID 중복이 있습니다!")
            return False

        # 어노테이션 ID 중복 체크
        ann_ids = [ann['id'] for ann in merged_data['annotations']]
        if len(ann_ids) != len(set(ann_ids)):
            print("경고: 어노테이션 ID 중복이 있습니다!")
            return False

        # 카테고리 ID 중복 체크
        cat_ids = [cat['id'] for cat in merged_data['categories']]
        if len(cat_ids) != len(set(cat_ids)):
            print("경고: 카테고리 ID 중복이 있습니다!")
            return False

        # 참조 무결성 체크
        valid_image_ids = set(img['id'] for img in merged_data['images'])
        valid_cat_ids = set(cat['id'] for cat in merged_data['categories'])

        invalid_refs = 0
        for ann in merged_data['annotations']:
            if ann['image_id'] not in valid_image_ids:
                print(f"경고: 존재하지 않는 image_id 참조: {ann['image_id']}")
                invalid_refs += 1
            if ann['category_id'] not in valid_cat_ids:
                print(f"경고: 존재하지 않는 category_id 참조: {ann['category_id']}")
                invalid_refs += 1

        if invalid_refs == 0:
            print("✓ 데이터 무결성 검증 통과!")
            return True
        else:
            print(f"경고: {invalid_refs}개의 참조 오류가 있습니다!")
            return False

    def create_train_val_split(self, merged_data, val_ratio=0.2):
        """
        훈련/검증 데이터 분할
        """
        import random

        images = merged_data['images'].copy()
        annotations = merged_data['annotations']
        categories = merged_data['categories']

        # 이미지 셔플
        random.seed(42)
        random.shuffle(images)

        # 분할
        total_images = len(images)
        val_size = int(total_images * val_ratio)

        val_images = images[:val_size]
        train_images = images[val_size:]

        # 이미지 ID 집합
        train_image_ids = set(img['id'] for img in train_images)
        val_image_ids = set(img['id'] for img in val_images)

        # 어노테이션 분할
        train_annotations = [ann for ann in annotations if ann['image_id'] in train_image_ids]
        val_annotations = [ann for ann in annotations if ann['image_id'] in val_image_ids]

        # 데이터셋 구성
        train_data = {
            "images": train_images,
            "annotations": train_annotations,
            "categories": categories,
            "info": merged_data['info']
        }

        val_data = {
            "images": val_images,
            "annotations": val_annotations,
            "categories": categories,
            "info": merged_data['info']
        }

        print(f"\n데이터 분할 완료:")
        print(f"  훈련: 이미지 {len(train_images)}개, 어노테이션 {len(train_annotations)}개")
        print(f"  검증: 이미지 {len(val_images)}개, 어노테이션 {len(val_annotations)}개")

        return train_data, val_data

    def copy_images_to_output(self):
        """
        이미지 파일들을 출력 디렉토리로 복사
        """
        print("이미지 파일들을 복사하는 중...")

        # 훈련 이미지 복사
        train_output_dir = os.path.join(self.output_dir, "images")
        os.makedirs(train_output_dir, exist_ok=True)

        copied_count = 0
        if os.path.exists(self.train_images_path):
            for img_file in os.listdir(self.train_images_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    src = os.path.join(self.train_images_path, img_file)
                    dst = os.path.join(train_output_dir, img_file)
                    shutil.copy2(src, dst)
                    copied_count += 1

        print(f"훈련 이미지 {copied_count}개 복사 완료")

        # 테스트 이미지 복사
        if os.path.exists(self.test_images_path):
            test_output_dir = os.path.join(self.output_dir, "test_images")
            os.makedirs(test_output_dir, exist_ok=True)

            test_count = 0
            for img_file in os.listdir(self.test_images_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    src = os.path.join(self.test_images_path, img_file)
                    dst = os.path.join(test_output_dir, img_file)
                    shutil.copy2(src, dst)
                    test_count += 1

            print(f"테스트 이미지 {test_count}개 복사 완료")

    def save_annotations(self, train_data, val_data, full_data):
        """
        COCO 포맷 어노테이션 파일 저장
        """
        # 전체 데이터
        with open(os.path.join(self.output_dir, "annotations_full.json"), 'w', encoding='utf-8') as f:
            json.dump(full_data, f, ensure_ascii=False, indent=2)

        # 훈련 데이터
        with open(os.path.join(self.output_dir, "annotations_train.json"), 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=2)

        # 검증 데이터
        with open(os.path.join(self.output_dir, "annotations_val.json"), 'w', encoding='utf-8') as f:
            json.dump(val_data, f, ensure_ascii=False, indent=2)

        print("\n어노테이션 파일 저장 완료:")
        print(f"  - annotations_full.json: 전체 데이터")
        print(f"  - annotations_train.json: 훈련 분할")
        print(f"  - annotations_val.json: 검증 분할")

    def generate_dataset_statistics(self, data):
        """
        데이터셋 통계 생성
        """
        print("\n=== 데이터셋 통계 ===")
        print(f"총 이미지 수: {len(data['images'])}")
        print(f"총 어노테이션 수: {len(data['annotations'])}")
        print(f"총 카테고리 수: {len(data['categories'])}")

        # 카테고리별 통계
        category_counts = defaultdict(int)
        category_names = {cat['id']: cat['name'] for cat in data['categories']}

        for ann in data['annotations']:
            category_counts[ann['category_id']] += 1

        print(f"\n카테고리별 어노테이션 수 (알약 종류별 인스턴스 수):")
        for cat_id, count in sorted(category_counts.items(), key=lambda x: x[1], reverse=True):
            cat_name = category_names.get(cat_id, f"Unknown({cat_id})")
            print(f"  {cat_name}: {count}개 인스턴스")

        # 이미지당 평균 객체 수
        if data['images']:
            avg_objects_per_image = len(data['annotations']) / len(data['images'])
            print(f"\n이미지당 평균 객체 수: {avg_objects_per_image:.2f}개")

        # 이미지 크기 통계
        if data['images']:
            widths = [img['width'] for img in data['images'] if 'width' in img]
            heights = [img['height'] for img in data['images'] if 'height' in img]

            if widths and heights:
                print(f"\n이미지 크기 통계:")
                print(f"  너비: 최소 {min(widths)}, 최대 {max(widths)}, 평균 {sum(widths)/len(widths):.1f}")
                print(f"  높이: 최소 {min(heights)}, 최대 {max(heights)}, 평균 {sum(heights)/len(heights):.1f}")

    def process_dataset(self, create_splits=True, val_ratio=0.2):
        """
        전체 전처리 파이프라인 실행
        """
        print("=== COCO 포맷 알약 데이터셋 병합 시작 ===")

        # 1. JSON 파일들 찾기
        json_files = self.find_all_json_files()

        if not json_files:
            print("JSON 파일을 찾을 수 없습니다!")
            return None

        # 2. COCO 어노테이션 병합
        merged_data = self.merge_coco_annotations(json_files)

        # 3. 데이터 무결성 검증
        if not self.validate_merged_data(merged_data):
            print("데이터 무결성 검증 실패!")
            return None

        # 4. 이미지 파일 복사
        self.copy_images_to_output()

        # 5. 훈련/검증 분할 (옵션)
        if create_splits and len(merged_data['images']) > 1:
            train_data, val_data = self.create_train_val_split(merged_data, val_ratio)
            self.save_annotations(train_data, val_data, merged_data)
        else:
            # 분할 없이 전체 데이터만 저장
            with open(os.path.join(self.output_dir, "annotations.json"), 'w', encoding='utf-8') as f:
                json.dump(merged_data, f, ensure_ascii=False, indent=2)
            print("전체 데이터를 annotations.json으로 저장했습니다.")

        # 6. 통계 생성
        self.generate_dataset_statistics(merged_data)

        print(f"\n병합 완료! 결과는 '{self.output_dir}' 디렉토리에 저장되었습니다.")

        return merged_data

## 메인 실행 함수

In [ ]:
def main():
    preprocessor = DETR_preprocessor(data_path = "./data/datasets/dataset_nano")

    # 전체 병합 실행
    dataset = preprocessor.process_dataset(
        create_splits=True,  # 훈련/검증 분할 여부
        val_ratio=0.2       # 검증 데이터 비율 (20%)
    )
    print("\n처리된 파일 구조:")
    print("./data/processed/")
    print("├── images/                    # 모든 훈련 이미지 파일")
    print("├── test_images/               # 테스트 이미지 (있는 경우)")
    print("├── annotations_full.json     # 전체 데이터")
    print("├── annotations_train.json    # 훈련 분할")
    print("└── annotations_val.json      # 검증 분할")

In [ ]:
if __name__ == "__main__":
    main()

=== COCO 포맷 알약 데이터셋 병합 시작 ===
총 148개의 JSON 파일을 찾았습니다.
COCO 포맷 JSON 파일들을 병합하는 중...
처리 중: K-002483-004378-023223-025438_0_2_0_2_70_000_200.json (1/148)
처리 중: K-003351-033880-038162_0_2_0_2_75_000_200.json (2/148)
처리 중: K-003351-016262-018147_0_2_0_2_70_000_200.json (3/148)
처리 중: K-003483-020238-025469-031885_0_2_0_2_90_000_200.json (4/148)
처리 중: K-003483-019861-022347-035206_0_2_0_2_90_000_200.json (5/148)
처리 중: K-003483-016262-027777-031885_0_2_0_2_70_000_200.json (6/148)
처리 중: K-003483-016232-019861-028763_0_2_0_2_75_000_200.json (7/148)
처리 중: K-003351-022074-032310_0_2_0_2_75_000_200.json (8/148)
처리 중: K-002483-005094-013395-023223_0_2_0_2_70_000_200.json (9/148)
처리 중: K-001900-016548-027926-044199_0_2_0_2_70_000_200.json (10/148)
처리 중: K-001900-016548-027926-044199_0_2_0_2_90_000_200.json (11/148)
처리 중: K-003351-020014-029667_0_2_0_2_90_000_200.json (12/148)
처리 중: K-001900-016551-024850-031705_0_2_0_2_75_000_200.json (13/148)
처리 중: K-003483-027777-029667-030308_0_2_0_2_90_000_200.jso

## 수행 시간
1. NANO: 8분

In [ ]:
# 전역 설정 (권장) - 이 Colab 세션에서 계속 사용
!git config --global user.name "Dongjin-1203"
!git config --global user.email "hambur1203@gmail.com"

In [ ]:
!git add .

In [ ]:
!git commit -m "DETR모델을 위한 전처리 파일 생성 및 새로운 전처리 코드 노트북 작성"

[main 3b74b1c] DETR모델을 위한 전처리 파일 생성 및 새로운 전처리 코드 노트북 작성
 3 files changed, 3 insertions(+), 1 deletion(-)
 create mode 100644 data_pipeline/notebooks/data_preprocesss_for_DETR.ipynb
 create mode 100644 notebooks/New_split_dataset.ipynb


In [ ]:
!git pull origin main

From https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
!git push origin main

Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 14.70 KiB | 537.00 KiB/s, done.
Total 8 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Dongjin-1203/Codeit_AI_4th_Drug_image_CV_project.git
   e460a65..3b74b1c  main -> main
